In [ ]:
# !pip install requests
import requests, os, json
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

# print("my environment variables are:")
# print(os.getenv('ASI_ONE_API_KEY'))
url = "https://api.asi1.ai/v1/chat/completions"
headers = {
"Authorization": f"Bearer {os.getenv('ASI_ONE_API_KEY')}",
"Content-Type": "application/json"
}
body = {
"model": "asi1-mini",
"messages": [{"role": "user", "content": "Hello! How can you help me today?"}]
}

# Check if API key exists before making the request
api_key = os.getenv('ASI_ONE_API_KEY')
if api_key:
    response = requests.post(url, headers=headers, json=body)
    result = response.json()
    
    # Better error handling for the API response
    if "choices" in result:
        print(result["choices"][0]["message"]["content"])
    else:
        print("Error in API response:", result)
else:
    print("ASI_ONE_API_KEY not found. Please check your .env file.")
import requests

# ASI:One API settings
API_KEY = os.getenv('ASI_ONE_API_KEY')
BASE_URL = "https://api.asi1.ai/v1"



In [ ]:

def get_weather(latitude, longitude):
    response = requests.get(f"https://api.open-meteo.com/v1/forecast?latitude={latitude}&longitude={longitude}&current=temperature_2m,wind_speed_10m&hourly=temperature_2m,relative_humidity_2m,wind_speed_10m")
    data = response.json()
    return data['current']['temperature_2m']


# Define the get_weather tool
get_weather_tool = {
    "type": "function",
    "function": {
        "name": "get_weather",
        "description": "Get current temperature for a given location (latitude and longitude).",
        "parameters": {
            "type": "object",
            "properties": {
                "latitude": {"type": "number"},
                "longitude": {"type": "number"}
            },
            "required": ["latitude", "longitude"]
        }
    }
}


# To this glorious masterpiece:
initial_message = [{
    "role": "user", 
    "content": "What's the current weather like in latitude 21 and longitude 31 right now?"
}]

# OR even better, because we're professionals (sort of):
messages_history = [{
    "role": "user",
    "content": "What's the current weather like in latitude 21 and longitude 31 right now?"
}]

payload = {
    "model": "asi1-mini",
    "messages": messages_history,  # Now it's a list, you beautiful disaster
    "tools": [get_weather_tool],
    "temperature": 0.7,
    "max_tokens": 1024
}


first_response = requests.post(
    "https://api.asi1.ai/v1/chat/completions",
    headers=headers,
    json=payload
)
print("calling first response.......")
print(first_response.json())

In [ ]:
first_response.raise_for_status()
first_response_json = first_response.json()

tool_calls = first_response_json["choices"][0]["message"].get("tool_calls", [])
print("tolls cals : ")
print(tool_calls)
messages_history = [
    initial_message,
    first_response_json["choices"][0]["message"]
]


In [ ]:
# Send tool results back to model for final response
if 'messages_history' in locals() and len(messages_history) > 2:
    final_payload = {
        "model": "asi1-mini",
        "messages": messages_history,
        "temperature": 0.7,
        "max_tokens": 1024
    }

    final_response = requests.post(
        f"{BASE_URL}/chat/completions",
        headers=headers,
        json=final_payload
    )

    if final_response.status_code == 200:
        final_response_json = final_response.json()
        final_answer = final_response_json["choices"][0]["message"]["content"]
        print("Final AI Response:")
        print("=" * 50)
        print(final_answer)
    else:
        print(f"Final request failed with status code: {final_response.status_code}")
        print("Error:", final_response.text)
else:
    print("No messages history available. Please run the previous cells first.")